# Data Cleaning

note: multiply AGI by 1000

In [1]:
# imports
import os 
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns

In [2]:
salary_df = pd.read_csv("../data/clean_data/cnc_sal_data.csv", infer_datetime_format= True, parse_dates= True)

In [3]:
salary_df["returns"]= salary_df["NO. OF RETURNS"]
salary_df.drop(columns= "NO. OF RETURNS", inplace=True)
salary_df.head()

,YEAR,ZIP CODE,AGI,returns
0,2007,73301,7230,1798
1,2007,73949,1915,43
2,2007,75001,693056,7128
3,2007,75002,2002247,26669
4,2007,75006,1053748,21953


In [4]:
# salary_df.shape
# salary_df.dtypes

In [6]:
salary_df["avg_income"]= salary_df["AGI"]* 1000/ salary_df["returns"]
salary_df.head()

,YEAR,ZIP CODE,AGI,returns,avg_income
0,2007,73301,7230,1798,4021.134594
1,2007,73949,1915,43,44534.883721
2,2007,75001,693056,7128,97230.078563
3,2007,75002,2002247,26669,75077.693202
4,2007,75006,1053748,21953,48000.182207


In [7]:
avg_income_df = salary_df[["YEAR", "ZIP CODE", "avg_income"]]
avg_income_df.dropna(inplace=True)
avg_income_df.head()

<ipython-input-7-d7a7bf6f67b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avg_income_df.dropna(inplace=True)


,YEAR,ZIP CODE,avg_income
0,2007,73301,4021.134594
1,2007,73949,44534.883721
2,2007,75001,97230.078563
3,2007,75002,75077.693202
4,2007,75006,48000.182207


In [8]:
avg_income_df.to_csv(path_or_buf="../data/clean_data/avg_income.csv", index=False)